# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Creo los dataframes en base a los archivos que he subido al dbfs (El archivo xlsx lo he transformado a csv antes de subirlo)
df_precios = spark.read.option("header", "true") \
               .option("delimiter", ",") \
               .option("multiline", "true") \
               .csv("/FileStore/Caso_3/airbnb_price.csv")

df_tipos_de_habitacion = spark.read.option("header", "true") \
                    .option("delimiter", ",") \
                    .option("multiline", "true") \
                    .csv("/FileStore/Caso_3/airbnb_room_type.csv")

df_resenas = spark.read.csv('/FileStore/Caso_3/airbnb_last_review.tsv', header=True, sep='\t')

# Muestro las primeras 3 filas de cada DataFrame para comprobar que se lean bien
print("DataFrame de precios:")
df_precios.show(3)

print("\nDataFrame de tipos de habitaciones:")
df_tipos_de_habitacion.show(3)

print("\nDataFrame de reseñas:")
df_resenas.show(3)


# Obtengo las fechas de las primeras y últimas reseñas a partir de la columna convertida al formato correcto para evitar errores
df_resenas = df_resenas.withColumn("last_review", to_date(col("last_review"), "MMMM dd yyyy"))
revisado_por_primera_vez = df_resenas.select(min("last_review")).collect()[0][0]
revisado_por_ultima_vez = df_resenas.select(max("last_review")).collect()[0][0]


# Cuento cuántos anuncios son de habitaciones privadas
numero_de_habitaciones_privadas = df_tipos_de_habitacion.filter(col("room_type") == "Private room").count()

# Calculo el precio promedio de los anuncios y redondeo a dos decimales
precio_medio = df_precios.withColumn("price", regexp_replace(col("price"), "[^0-9.]", "").cast("float"))
precio_medio = precio_medio.select(round(avg("price"), 2).alias("avg_price")).collect()[0][0]

# Creo el df con todo lo obtenido antes para mostrarlo
df_final = spark.createDataFrame(
    [(revisado_por_primera_vez, revisado_por_ultima_vez, numero_de_habitaciones_privadas, avg_price)],
    ["first_reviewed", "last_reviewed", "nb_private_rooms", "avg_price"]
)


df_final.show(truncate=False)



DataFrame de precios:
+----------+-----------+--------------------+
|listing_id|      price|         nbhood_full|
+----------+-----------+--------------------+
|      2595|225 dollars|  Manhattan, Midtown|
|      3831| 89 dollars|Brooklyn, Clinton...|
|      5099|200 dollars|Manhattan, Murray...|
+----------+-----------+--------------------+
only showing top 3 rows


DataFrame de tipos de habitaciones:
+----------+--------------------+---------------+
|listing_id|         description|      room_type|
+----------+--------------------+---------------+
|      2595|Skylit Midtown Ca...|Entire home/apt|
|      3831|Cozy Entire Floor...|Entire home/apt|
|      5099|Large Cozy 1 BR A...|Entire home/apt|
+----------+--------------------+---------------+
only showing top 3 rows


DataFrame de reseñas:
+----------+-----------+------------+
|listing_id|  host_name| last_review|
+----------+-----------+------------+
|      2595|   Jennifer| May 21 2019|
|      3831|LisaRoxanne|July 05 2019|
|     